## Импорт библиотек

In [ ]:
import os
import gc
import pickle
import time

import numpy as np
import pandas as pd
import polars as pl

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import VotingRegressor

from joblib import dump
from joblib import load

import lightgbm as lgb
import catboost as cb

import optuna

import matplotlib.pyplot as plt

In [ ]:
# Время старта работы ноутбука
notebook_starttime = time.time()

In [ ]:
# Возвращает сколько уже работает ноутбук
def p_time():
    #
    run_time = round(time.time() - notebook_starttime)
    return str(run_time).zfill(5)+' sec:'

## Настройка: сабмит или локально

In [ ]:
# Ставим is_local в True, если локально работаем, если сабмитим - ставим в False
#is_local = False
is_local = True

# Ставим is_gpu в True, если будем работать на GPU, если на процессоре - ставим в False
# is_gpu = False
is_gpu = True

# Ставим is_tuning в True, если запускаем подбор гиперпараметров в Optuna
is_tuning = False

# Начальная дата обучения модели
training_start_date = 'datetime >= "2022-01-01 00:00:00"'

In [ ]:
class MonthlyKFold:
    def __init__(self, n_splits=3):
        self.n_splits = n_splits
        
    def split(self, X, y, groups=None):
        dates = 12 * X["year"] + X["month"]
        timesteps = sorted(dates.unique().tolist())
        X = X.reset_index()
        
        for t in timesteps[-self.n_splits:]:
            idx_train = X[dates.values < t].index
            idx_test = X[dates.values == t].index
            
            yield idx_train, idx_test
            
    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

## Feature Engineering sub

In [ ]:
def feature_eng(df_data, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days):
    working_days = (
        working_days
        .with_columns(
            pl.col("date").cast(pl.Date)
        )
    )
    
    df_data = (
        df_data
        .with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )
    )
    
    df_client = (
        df_client
        .with_columns(
            (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
        )
    )
    
    df_gas = (
        df_gas
        .rename({"forecast_date": "date"})
        .with_columns(
            (pl.col("date") + pl.duration(days=1)).cast(pl.Date)
        )
    )
    
    df_electricity = (
        df_electricity
        .rename({"forecast_date": "datetime"})
        .with_columns(
            pl.col("datetime") + pl.duration(days=1)
        )
    )
    
    df_location = (
        df_location
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32)
        )
    )
    
    df_forecast = (
        df_forecast
        .rename({"forecast_datetime": "datetime"})
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            #pl.col('datetime').dt.convert_time_zone("Europe/Bucharest").dt.replace_time_zone(None).cast(pl.Datetime("us")),
            pl.col('datetime').dt.replace_time_zone(None).cast(pl.Datetime("us"))
            #pl.col('datetime').cast(pl.Datetime)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_historical = (
        df_historical
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
            pl.col("datetime") + pl.duration(hours=37)
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_forecast_date = (
        df_forecast
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_forecast_local = (
        df_forecast
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    df_historical_date = (
        df_historical
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_historical_local = (
        df_historical
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    # Объединение всех обработанных данных с основным датафреймом df_data
    df_data = (
        df_data
        .join(df_gas, on="date", how="left")
        .join(df_client, on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_electricity, on="datetime", how="left")
        
        .join(df_forecast_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_local, on=["county", "datetime"], how="left", suffix="_fl")
        .join(df_historical_date, on="datetime", how="left", suffix="_hd")
        .join(df_historical_local, on=["county", "datetime"], how="left", suffix="_hl")
        
        .join(df_forecast_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fdw")
        .join(df_forecast_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_flw")
        .join(df_historical_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_hdw")
        .join(df_historical_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_hlw")
        
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        #.join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=8)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=9)).rename({"target": "target_8"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=10)).rename({"target": "target_9"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=11)).rename({"target": "target_10"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=12)).rename({"target": "target_11"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=13)).rename({"target": "target_12"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_13"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        #Добавляем коолонку рабочий день или нет.
        .join(working_days, on="date", how="left")
        # Создание категориальных признаков и тригонометрических функций времени
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"), # Добавление номера дня в году
            pl.col("datetime").dt.hour().alias("hour"),# Добавление часа
            pl.col("datetime").dt.day().alias("day"),# Добавление дня
            pl.col("datetime").dt.weekday().alias("weekday"),# Добавление дня недели
            pl.col("datetime").dt.month().alias("month"),# Добавление месяца
            pl.col("datetime").dt.year().alias("year"),# Добавление года
        )
        # Приведение типов данных
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("category_1"),
        )
        
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"), # Тригонометрические функции для дня в году
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )
        
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
         # Удаление ненужных колонок
        .drop("date", "datetime", "hour", "dayofyear")
    )
    
    # return df_data, df_historical_local
    return df_data

In [ ]:
def to_pandas(X, y=None):
    cat_cols = ["county", "is_business", "product_type", "is_consumption", "category_1"]
    
    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()    
    
    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")
    
    '''
    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)
    '''
    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)
    
    return df

In [ ]:
# для оптуны
# Диапазон гиперпараметров для модели обучающейся на данных is_consumption=1 
def lgb_objective_cons1(trial):
    params = {
        'device'            : 'gpu',
        'gpu_platform_id'   : 1,
        'gpu_device_id'     : 0,
        'n_estimators'      : trial.suggest_int('n_estimators', 1000, 2000),
        'verbose'           : -1,
        'random_state'      : 42,
        'objective'         : 'l2',
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1),
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode'  : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 20.0),
        'reg_lambda'        : trial.suggest_float('reg_lambda', 1e-2, 20.0),
        'min_child_samples' : trial.suggest_int('min_child_samples', 4, 256),
        'max_depth'         : trial.suggest_int('max_depth', 5, 20),
        'max_bin'           : trial.suggest_int('max_bin', 32, 200),
    }

    model  = lgb.LGBMRegressor(**params)
    X      = df_train[df_train['is_consumption']==1].drop(columns=["target", "datetime"]).reset_index(drop=True)
    y      = df_train[df_train['is_consumption']==1]["target"].reset_index(drop=True)
    scores = cross_val_score(model, X, y, groups=groups, cv=CV, scoring='neg_mean_absolute_error')
    
    return -np.mean(scores)

# Диапазон гиперпараметров для модели обучающейся на данных is_consumption=0
def lgb_objective_cons0(trial):
    params = {
        'device'            : 'gpu',
        'gpu_platform_id'   : 1,
        'gpu_device_id'     : 0,
        'n_estimators'      : trial.suggest_int('n_estimators', 1000, 2000),
        'verbose'           : -1,
        'random_state'      : 42,
        'objective'         : 'l2',
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1),
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode'  : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'reg_alpha'         : trial.suggest_float('reg_alpha', 1e-2, 20.0),
        'reg_lambda'        : trial.suggest_float('reg_lambda', 1e-2, 20.0),
        'min_child_samples' : trial.suggest_int('min_child_samples', 4, 256),
        'max_depth'         : trial.suggest_int('max_depth', 5, 20),
        'max_bin'           : trial.suggest_int('max_bin', 32, 200),
    }

    model  = lgb.LGBMRegressor(**params)
    X      = df_train[df_train['is_consumption']==0].drop(columns=["target", "datetime"]).reset_index(drop=True)
    y      = df_train[df_train['is_consumption']==0]["target"].reset_index(drop=True)
    scores = cross_val_score(model, X, y, groups=groups, cv=CV, scoring='neg_mean_absolute_error')
    
    return -np.mean(scores)

## Global Variables

In [ ]:
root = "/kaggle/input/predict-energy-behavior-of-prosumers"

# Для локальных вычислений. Последний data_block_id тренировочной выборки
# А начиная со следующего data_block_id и до конца идет тест
train_end_data_block_id = 500
# train_end_data_block_id = 600

data_cols        = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id', 'data_block_id']
# В df_data_cols колонки в таком порядке в каком они потом формируются в df_data
df_data_cols     = ['county', 'is_business', 'product_type', 'target', 'is_consumption', 'datetime', 'data_block_id', 'row_id']
client_cols      = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_cols         = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_cols = ['forecast_date', 'euros_per_mwh']
forecast_cols    = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_cols  = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols    = ['longitude', 'latitude', 'county']
target_cols      = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

save_path = None
load_path = None

## Исследование

In [ ]:
if is_local:
    # Загрузка данных об энергопотреблении
    train = pd.read_csv(os.path.join(root, "train.csv"))
    
    # Создание сводной таблицы с средними значениями целевой переменной (target)
    # для каждой комбинации даты, округа, типа продукта, бизнеса и потребления
    pivot_train = train.pivot_table(
        index='datetime',
        columns=['county', 'product_type', 'is_business', 'is_consumption'],
        values='target',
        aggfunc='mean'
    )
    
    # Переименование колонок для удобства доступа и интерпретации
    pivot_train.columns = ['county{}_productType{}_isBusiness{}_isConsumption{}'.format(*col) for col in pivot_train.columns.values]
    pivot_train.index = pd.to_datetime(pivot_train.index)
    
    pivot_train

### 2023 год 

In [ ]:
if is_local:
    # Копирование сводной таблицы для визуализации
    df_plot = pivot_train.copy()
    
    # Нормализация данных для визуализации
    df_plot = (df_plot - df_plot.min()) / (df_plot.max() - df_plot.min())
    
    # Ресемплирование данных по дням и вычисление средних значений
    df_plot_resampled_D = df_plot.resample('D').mean()
    
    # Визуализация нормализованных данных с прозрачностью (alpha=0.1)
    df_plot_resampled_D.loc['2022-7':].plot(alpha=0.1, color='green', figsize=(18, 6), legend=False)


In [ ]:
if is_local:
    # Выбор колонок, соответствующих различным категориям потребления
    columns_consumption_0 = df_plot_resampled_D.columns[df_plot_resampled_D.columns.str.contains('isConsumption0')]
    columns_consumption_1 = df_plot_resampled_D.columns[df_plot_resampled_D.columns.str.contains('isConsumption1')]
    
    # Создание фигуры для визуализации
    plt.figure(figsize=(15, 6))
    
    # Создание пустых линий для легенды
    plt.plot([], color='red', label='is_Consumption = 1')  # Изменено на желтый цвет
    plt.plot([], color='black', label='is_Consumption = 0')   # Изменено на черный цвет
    
    # Отображение легенды
    plt.legend()
    
    # Визуализация данных для 'is_Consumption = 0' черным цветом
    for column in columns_consumption_0:
        df_plot_resampled_D.loc['2022-7':, column].plot(alpha=0.1, color='black', legend=False)  # Изменено на черный
    
    # Визуализация данных для 'is_Consumption = 1' желтым цветом
    for column in columns_consumption_1:
        df_plot_resampled_D.loc['2022-7':, column].plot(alpha=0.1, color='red', legend=False)  # Изменено на желтый
    
    # Отображение графика
    plt.show()




## Подготовка данных

### Запись тестовых и тренировочных csv файлов

In [ ]:
if is_local:
    # Если выполняем локально
    train_path = 'train'
    if not os.path.exists(train_path):
        # Создание каталога, если его нет
        os.makedirs(train_path)
    test_path = 'example_test_files'
    if not os.path.exists(test_path):
        # Создание каталога, если его нет
        os.makedirs(test_path)
else:
    # Если сабмит
    train_path = root
# Путь, куда запишем csv файлы для теста

In [ ]:
# Разделяет датафрейм на тренировочную и тестовую часть
# Возвращает часть датафрейма для тренировки, тестовую часть датафрейма записывает в каталог с тестами
def split_train_test(filename):
    df = pd.read_csv(os.path.join(root, filename))
    
    #Запишем часть данных для теста
    test_df = df[df["data_block_id"] > train_end_data_block_id]
    if (filename =="train.csv"):
        # Берем только те ячейки где target был не нулевым
        test_df = test_df[test_df["target"].notnull()]
        
    test_df.to_csv(os.path.join(test_path, filename), index=False)

    #Запишем часть данных для трейна
    train_df = df[df["data_block_id"] <= train_end_data_block_id]
    train_df.to_csv(os.path.join(train_path, filename), index=False)

# Доводим до ума тестовые таблицы чтобы они были точно такие как в реальном сабмите
def test_dfs_tune():
    # Делаем таблицу revealed_targets.csv
    df = pd.read_csv(os.path.join(root, "train.csv"))
    df = df[df["data_block_id"] > train_end_data_block_id - 2]
    df["data_block_id"] += 2
    df = df[df["target"].notnull()]
    df.to_csv(os.path.join(test_path, 'revealed_targets.csv'), index=False)
    
    # Делаем таблицу test.csv
    df = pd.read_csv(os.path.join(test_path, "train.csv"))
    df.rename(columns={'datetime': 'prediction_datetime'}, inplace=True)
    df.drop('target', axis=1, inplace=True)
    df['currently_scored'] = False
    df.to_csv(os.path.join(test_path, 'test.csv'), index=False)
    
    # Делаем таблицу sample_submission.csv
    selected_columns = ['row_id', 'data_block_id']
    df = df[selected_columns]
    df['target'] = 0
    df.to_csv(os.path.join(test_path, 'sample_submission.csv'), index=False)

# Сборка разделения файлов
def make_split():
    # csv файлы которые будем делить:
    csv_names = ["train.csv", "client.csv", "gas_prices.csv", "electricity_prices.csv", "forecast_weather.csv", "historical_weather.csv"]
    for csv_name in csv_names:
        split_train_test(csv_name)
    # Доделываем тестовые таблицы
    test_dfs_tune()

In [ ]:
%%time

# Создаем файлы csv c тренировочными и тестовыми таблицами
if is_local:
    # Пока отключил создание тестовых файлом. У меня локально они есть
    # make_split()
    pass

### Data I/O

In [ ]:
%%time
df_data        = pl.read_csv(os.path.join(train_path, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client      = pl.read_csv(os.path.join(train_path, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas         = pl.read_csv(os.path.join(train_path, "gas_prices.csv"), columns=gas_cols, try_parse_dates=True)
df_electricity = pl.read_csv(os.path.join(train_path, "electricity_prices.csv"), columns=electricity_cols, try_parse_dates=True)
df_forecast    = pl.read_csv(os.path.join(train_path, "forecast_weather.csv"), columns=forecast_cols, try_parse_dates=True)
df_historical  = pl.read_csv(os.path.join(train_path, "historical_weather.csv"), columns=historical_cols, try_parse_dates=True)
#df_location    = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
df_location    = pl.read_csv('/kaggle/input/locations/county_lon_lats.csv', columns=location_cols, try_parse_dates=True)
df_target      = df_data.select(target_cols)
working_days   = pl.read_csv('/kaggle/input/working-days/working_days.csv', try_parse_dates=True)

schema_data        = df_data.schema
schema_client      = df_client.schema
schema_gas         = df_gas.schema
schema_electricity = df_electricity.schema
schema_forecast    = df_forecast.schema
schema_historical  = df_historical.schema
schema_target      = df_target.schema

### HyperParam Optimization

In [ ]:
# Подбор гиперпараметров для модели is_consumption=1
if is_tuning:
    X, y = df_data.drop("target"), df_data.select("target")
    X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
    df_train = to_pandas(X, y)

    df_train = df_train[df_train["target"].notnull()].query(training_start_date)
    
    study = optuna.create_study(direction='minimize', study_name='Regressor')
    study.optimize(lgb_objective_cons1, n_trials=100, show_progress_bar=True)
pass

### Validation

In [ ]:
'''result = cross_validate(
    estimator=lgb.LGBMRegressor(**best_params, random_state=42),
    X=df_train.drop(columns=["target"]), 
    y=df_train["target"],
    scoring="neg_mean_absolute_error",
    cv=MonthlyKFold(1),
)

print(f"Fit Time(s): {result['fit_time'].mean():.3f}")
print(f"Score Time(s): {result['score_time'].mean():.3f}")
print(f"Error(MAE): {-result['test_score'].mean():.3f}")'''
pass

### Training

#### Параметры для LGB is_consumption=1

In [ ]:
# Тестируем на GPU
if is_gpu:
    p1={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1953, 'learning_rate': 0.06292846025674187, 'colsample_bytree': 0.885312594375526, 'colsample_bynode': 0.7831589504736596, 'reg_alpha': 7.677890737474055, 'reg_lambda': 16.04294344413536, 'min_child_samples': 69, 'max_depth': 13, 'max_bin': 122}
    p2={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1781, 'learning_rate': 0.054763617969941024, 'colsample_bytree': 0.9233976730768843, 'colsample_bynode': 0.8969396545503117, 'reg_alpha': 9.893144785987541, 'reg_lambda': 19.956571072303618, 'min_child_samples': 30, 'max_depth': 11, 'max_bin': 128}
    p3={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1713, 'learning_rate': 0.053743689746779025, 'colsample_bytree': 0.8813612023144495, 'colsample_bynode': 0.8739181786588937, 'reg_alpha': 6.967728410825889, 'reg_lambda': 18.772923941576302, 'min_child_samples': 8, 'max_depth': 11, 'max_bin': 130}
    p4={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1852, 'learning_rate': 0.0592383731526721, 'colsample_bytree': 0.9158803696144284, 'colsample_bynode': 0.8677708234107011, 'reg_alpha': 8.178421543960194, 'reg_lambda': 18.453388032680913, 'min_child_samples': 7, 'max_depth': 13, 'max_bin': 127}
    p5={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1854, 'learning_rate': 0.06206288936616207, 'colsample_bytree': 0.8880309956381198, 'colsample_bynode': 0.8902254554150658, 'reg_alpha': 8.741757862603745, 'reg_lambda': 16.131881076970146, 'min_child_samples': 11, 'max_depth': 13, 'max_bin': 133}
    p6={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1760, 'learning_rate': 0.05058624430058491, 'colsample_bytree': 0.8623393986024189, 'colsample_bynode': 0.9242352457544074, 'reg_alpha': 10.403491499807716, 'reg_lambda': 17.336966880450976, 'min_child_samples': 16, 'max_depth': 12, 'max_bin': 131}
    p7={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1783, 'learning_rate': 0.053984137286090667, 'colsample_bytree': 0.7988556940541043, 'colsample_bynode': 0.8962066698736209, 'reg_alpha': 10.033954743421075, 'reg_lambda': 19.869585514651835, 'min_child_samples': 29, 'max_depth': 11, 'max_bin': 128}
else:
    pass

#### Параметры для LGB is_consumption=0

In [ ]:
if is_gpu:
    # Параметры для catboost c GPU
    c1={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1928, 'learning_rate': 0.07746737348328141, 'colsample_bytree': 0.78312363495825, 'colsample_bynode': 0.519093615242415, 'reg_alpha': 3.5475701282438967, 'reg_lambda': 17.88655785653258, 'min_child_samples': 20, 'max_depth': 17, 'max_bin': 37}
    c2={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1881, 'learning_rate': 0.07864103296902626, 'colsample_bytree': 0.7910482238038328, 'colsample_bynode': 0.48895580052374654, 'reg_alpha': 7.375666493081315, 'reg_lambda': 15.347283246448637, 'min_child_samples': 15, 'max_depth': 15, 'max_bin': 41}
    c3={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1959, 'learning_rate': 0.06577095971205155, 'colsample_bytree': 0.8648745524121579, 'colsample_bynode': 0.4208704631805802, 'reg_alpha': 7.334717669004279, 'reg_lambda': 19.53863883138691, 'min_child_samples': 17, 'max_depth': 15, 'max_bin': 37}
    c4={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1962, 'learning_rate': 0.06171792620856953, 'colsample_bytree': 0.9584819437986425, 'colsample_bynode': 0.5405402088572034, 'reg_alpha': 3.307191570022457, 'reg_lambda': 16.209300259712922, 'min_child_samples': 43, 'max_depth': 19, 'max_bin': 40}
    c5={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1900, 'learning_rate': 0.07441152178622895, 'colsample_bytree': 0.8801531103073503, 'colsample_bynode': 0.337510749744266, 'reg_alpha': 6.105532634402442, 'reg_lambda': 19.489948983222703, 'min_child_samples': 28, 'max_depth': 11, 'max_bin': 51}
    c6={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1923, 'learning_rate': 0.07413128975186019, 'colsample_bytree': 0.9108240472209401, 'colsample_bynode': 0.368498860839913, 'reg_alpha': 5.928691934278827, 'reg_lambda': 19.535434444939284, 'min_child_samples': 35, 'max_depth': 16, 'max_bin': 43}
    c7={'device': 'gpu', 'verbose': -1, 'random_state': 42, 'objective': 'l2', 'n_estimators': 1899, 'learning_rate': 0.07536122429341643, 'colsample_bytree': 0.8755415544495265, 'colsample_bynode': 0.3971515541568271, 'reg_alpha': 7.825234820629166, 'reg_lambda': 19.539558271427442, 'min_child_samples': 16, 'max_depth': 10, 'max_bin': 67}
else:
    pass

#### Класс моделей

In [ ]:
# Класс обертка для моделей. Хранит различные пареметры для обучения моделей.
# Например диапазоны данных на которых учить модель, как часто обучать заново и другие параметры
class Models:
    
    # Инициализирует параметры обучения
    # init_model - готовый объект модели для обучения
    def __init__(self):
        # Инициализируем словари
        # Ключами во всех словарях будет имя модели
        
        # Словарь с моделями
        self.models =  dict()
        
        # Словарь с описанием периодов обучения модели
        # пока это матрица. из двух столбцов и двух строк в каждой строке описание периода
        # первая колонка на сколько data_block_id в конеце обучени отстоит от доступного конца данных
        # вторая колонка сколько data_block_id будет в периоде на котором обучаемся.
        # data_block_id могут быть эквивалентны дням, но могут и отличаться, если данные будут подавать блоками не равными дням
        # либо если будут разрывы в данных. Но они точно будут эквиваленты циклам предсказания
        self.data_block_id_intervals  = dict()

        # Словарь с указанием по какой минимальный data_block_id отрезать данные.
        # То есть меньше data_block_id_min не берем данные для обучения в любом случае, чтобы не было определено в data_block_id_intervals
        self.data_block_id_min  = dict()
        
        # Если 1, то модель предназначена для предсказания потребления электричества
        # Если 0, то модель предназначена для предсказания производства электричества
        self.is_consumption = dict()
        
        # Раз во сколько иттераций обучат модель
        self.learn_again_period = dict()
        
        # Смещение для начала обучения модели. Добавляется к номеру итерации сабмита.
        # Скажем если смещение 6 номер итерации 1, а обучаемся раз в чем итераций. То обучение будет в первуже итерацию сабмита.
        self.learn_again_offset = dict()
        
        # Время в секундах сколько заняло последнее обучение модели
        self.last_learn_time = dict()
    
    # Добавляет еще одну модель
    # model_name - название модели
    # new_model - объект модели
    def add_model(self, model_name, new_model, is_consumption, data_block_id_intervals,
                  data_block_id_min, learn_again_period, learn_again_offset):
        
        self.models[model_name] = new_model
        self.is_consumption[model_name] = is_consumption
        self.data_block_id_intervals[model_name] = data_block_id_intervals
        self.data_block_id_min[model_name] = data_block_id_min
        self.learn_again_period[model_name] = learn_again_period
        self.learn_again_offset[model_name] = learn_again_offset
        self.last_learn_time[model_name] = 0
        
    # Обучает модель
    # model_name - название модели
    # df_train - датафрейм который содержит данные для обучения и целевой признак
    def fit_one_model(self, model_name, df_train):
        print(p_time(), 'fit model:', model_name)
        self.models[model_name].fit(
            X=df_train.drop(columns=["target", "data_block_id"]),
            y=df_train["target"]
        )
    
    # Делает предсказания от отдельной модели.
    # model_name - название модели
    # X - признаки на которых нужно сделать предсказание
    def predict_one_model(self, model_name, X):
        print(p_time(), 'predict model:', model_name)
        y = (self.models[model_name]
             .predict(X.drop(columns=["data_block_id"])).clip(0)
            )
        return(y)
    
    # Обучает все добавленные модели для которых наступило время их обучения
    # df_train - датафрейм который содержит данные для обучения и целевой признак
    # itter_n - номер иттерации в сабмите.
    # если itter_n равен <=0. Значит первоначальное обучение и обучаем всем модели
    def fit(self, df_train, itter_n):
        max_block_id = df_train["data_block_id"].max()
        # Перебираем все модели
        for m_name in self.learn_again_period:
            # Либо сказали все модели учить
            if ((itter_n <= 0)
                # Либо учим если номер итераиции в сабмите плюс смещение делится на цело на период обучения
                or (((itter_n + self.learn_again_offset[m_name])
                     % self.learn_again_period[m_name]) == 0)):
                
                d_i = self.data_block_id_intervals[m_name]
                
                # Выделяем данные для обучения
                df_train_int = df_train[
                    # До какого data_block_id учим первый блок
                    (((df_train['data_block_id']<=max_block_id-d_i[0][0])
                     # C какого data_block_id учим первый блок
                     &(df_train['data_block_id']>(max_block_id-d_i[0][0]-d_i[0][1])))
                    # До какого data_block_id учим второй блок
                    |((df_train['data_block_id']<=max_block_id-d_i[1][0])
                      # С какого data_block_id учим второй блок
                      &(df_train['data_block_id']>(max_block_id-d_i[1][0]-d_i[1][1]))))
                #  выбираем только те данные которые больше data_block_id_min
                &(df_train['data_block_id']>=self.data_block_id_min[m_name])
                #  выбираем только те данные для обучения по is_consumption на которых специализируетсмя модель
                &(df_train['is_consumption']==self.is_consumption[m_name])
                # Оставляем только notnull таргеты
                &(df_train["target"].notnull())
                ]
                
                # Обучаем модель
                self.fit_one_model(m_name, df_train_int)
    
    # Делает предсказание всеми добавленными моделями и сводит их в одно предсказание
    def predict(self, X):
        # Создаем датафрейм для предсказаний
        # Первый столбец содержит информацию 'is_consumption'
        # Для каждой модели отдельный столбец с предсказаниями
        # В predict_df_0 будут предсказания для моделей с is_consumption == 0
        predict_df_0 = pd.DataFrame(X['is_consumption'])
        # В predict_df_1 будут предсказания для моделей с is_consumption == 1
        predict_df_1 = pd.DataFrame(X['is_consumption'])
        
        # Перебираем все модели
        for m_name in self.learn_again_period:
            # Делаем предсказание в соответвующий датафрейм в столбец модели
            if self.is_consumption[m_name] == 0:
                predict_df_0[m_name] = self.predict_one_model(m_name, X).clip(0)
            else:
                predict_df_1[m_name] = self.predict_one_model(m_name, X).clip(0)
                
        # Суммируем предсказания моделей раздельно по датафреймам
        predict_df_0['target'] = predict_df_0.iloc[:, 1:].mean(axis=1)
        predict_df_1['target'] = predict_df_1.iloc[:, 1:].mean(axis=1)
        
        # Сведение в одно предсказание потребления и производства электричества у просьюмеров
        predict_df = predict_df_1[['is_consumption', 'target']]

        predict_df.loc[predict_df['is_consumption']==0, 'target'] = predict_df_0.loc[predict_df_0['is_consumption']==0, 'target']
        
        return predict_df['target'].values

In [ ]:
models = Models()

In [ ]:
# Добавляем модель для потредление электричества
'''
models.add_model(
    model_name = 'model',
    new_model = lgb.LGBMRegressor(**p1, verbosity=-1, random_state=42),
    is_consumption = 1,
    data_block_id_intervals = [[0,30],[305,90]],
    data_block_id_min = 122,
    learn_again_period = 7,
    learn_again_offset = 0
)

# Добавляем модель для генерации электричества
models.add_model(
    model_name = 'model_solar',
    new_model = lgb.LGBMRegressor(**p1, verbosity=-1, random_state=42),
    is_consumption = 0,
    data_block_id_intervals = [[0,30],[305,90]],
    data_block_id_min = 122,
    learn_again_period = 7,
    learn_again_offset = 0
)
'''

model = VotingRegressor([
    ('lgb_1', lgb.LGBMRegressor(**p1)), 
    ('lgb_2', lgb.LGBMRegressor(**p2)), 
    ('lgb_3', lgb.LGBMRegressor(**p3)), 
    ('lgb_4', lgb.LGBMRegressor(**p4)), 
    ('lgb_5', lgb.LGBMRegressor(**p5)),
    #('lgb_6', lgb.LGBMRegressor(**p6)),
    #('lgb_7', lgb.LGBMRegressor(**p7)),  
])

model_solar = VotingRegressor([
    ('lgb_1', lgb.LGBMRegressor(**c1)), 
    ('lgb_2', lgb.LGBMRegressor(**c2)), 
    ('lgb_3', lgb.LGBMRegressor(**c3)), 
    ('lgb_4', lgb.LGBMRegressor(**c4)), 
    ('lgb_5', lgb.LGBMRegressor(**c5)),
    #('lgb_6', lgb.LGBMRegressor(**c6)),
    #('lgb_7', lgb.LGBMRegressor(**c7)),  
])

models.add_model(
    model_name = 'model',
    new_model = model,
    is_consumption = 1,
    data_block_id_intervals = [[0,1000],[0,0]],
    data_block_id_min = 0,
    learn_again_period = 2,
    learn_again_offset = 0
)

# Добавляем модель для генерации электричества
models.add_model(
    model_name = 'model_solar',
    new_model = model_solar,
    is_consumption = 0,
    data_block_id_intervals = [[0,1000],[0,0]],
    data_block_id_min = 0,
    learn_again_period = 2,
    learn_again_offset = 0
)

'''
models.add_model(
    model_name = 'model',
    new_model = lgb.LGBMRegressor(**p1, verbosity=-1, random_state=42),
    is_consumption = 1,
    data_block_id_intervals = [[0,90],[305,120]],
    data_block_id_min = 122,
    learn_again_period = 7,
    learn_again_offset = 0
)

# Добавляем модель для генерации электричества
models.add_model(
    model_name = 'model_solar',
    new_model = lgb.LGBMRegressor(**p1, verbosity=-1, random_state=42),
    is_consumption = 0,
    data_block_id_intervals = [[0,90],[305,120]],
    data_block_id_min = 122,
    learn_again_period = 7,
    learn_again_offset = 0
)
'''
pass

#### Обучение моделей

In [ ]:
#if is_local:
#    dump(model_solar, 'model_solar.joblib')
#    dump(model, 'model_lgbm.joblib')

Расскоментировать при необходимости загрузку ранее сохраненных моделей

In [ ]:
# Для загрузки локально
#model_solar = load('model_solar.joblib')
#model = load('model_lgbm.joblib')

# Для загрузки на kaggle
#model_solar = load('/kaggle/input/enefit/model_solar.joblib')
#model = load('/kaggle/input/enefit/model_lgbm.joblib')

## Prediction

In [ ]:
if is_local:
    # Если выполняем локально, а не сабмитим на кагл,
    # то выбираем другое имя для файла submission.csv.
    # Потому что в submission.csv записать прав нет и вылетает по ошибке
    submission_name = 'submission_loc.csv'
else:
    submission_name = 'submission.csv'

### Содержимое public_timeseries_testing_util.py

С необходимыми праками. Решил не импортировать его. а прямо тут. Так удобнее переносить на kaggle

In [ ]:
'''
An unlocked version of the timeseries API intended for testing alternate inputs.
Mirrors the production timeseries API in the crucial respects, but won't be as fast.

ONLY works afer the first three variables in MockAPI.__init__ are populated.
'''

from typing import Sequence, Tuple


class MockApi:
    def __init__(self):
        '''
        YOU MUST UPDATE THE FIRST THREE LINES of this method.
        They've been intentionally left in an invalid state.

        Variables to set:
            input_paths: a list of two or more paths to the csv files to be served
            group_id_column: the column that identifies which groups of rows the API should serve.
                A call to iter_test serves all rows of all dataframes with the current group ID value.
            export_group_id_column: if true, the dataframes iter_test serves will include the group_id_column values.
        '''
        self.input_paths: Sequence[str] = ['example_test_files/test.csv',
                                   'example_test_files/revealed_targets.csv', 
                                   'example_test_files/client.csv',
                                   'example_test_files/historical_weather.csv',
                                   'example_test_files/forecast_weather.csv',
                                   'example_test_files/electricity_prices.csv',
                                   'example_test_files/gas_prices.csv',
                                   'example_test_files/sample_submission.csv']
        self.group_id_column: str = 'data_block_id'
        self.export_group_id_column: bool = False
        # iter_test is only designed to support at least two dataframes, such as test and sample_submission
        assert len(self.input_paths) >= 2

        self._status = 'initialized'
        self.predictions = []

    def iter_test(self) -> Tuple[pd.DataFrame]:
        '''
        Loads all of the dataframes specified in self.input_paths,
        then yields all rows in those dataframes that equal the current self.group_id_column value.
        '''
        if self._status != 'initialized':

            raise Exception('WARNING: the real API can only iterate over `iter_test()` once.')

        dataframes = []
        for pth in self.input_paths:
            dataframes.append(pd.read_csv(pth, low_memory=False))
        group_order = dataframes[0][self.group_id_column].drop_duplicates().tolist()
        dataframes = [df.set_index(self.group_id_column) for df in dataframes]

        for group_id in group_order:
            self._status = 'prediction_needed'
            current_data = []
            for df in dataframes:
                cur_df = df.loc[group_id].copy()
                # returning single line dataframes from df.loc requires special handling
                if not isinstance(cur_df, pd.DataFrame):
                    cur_df = pd.DataFrame({a: b for a, b in zip(cur_df.index.values, cur_df.values)}, index=[group_id])
                    cur_df.index.name = self.group_id_column
                cur_df = cur_df.reset_index(drop=not(self.export_group_id_column))
                current_data.append(cur_df)
            yield tuple(current_data)

            while self._status != 'prediction_received':
                print('You must call `predict()` successfully before you can continue with `iter_test()`', flush=True)
                yield None

        with open(submission_name, 'w') as f_open:
            pd.concat(self.predictions).to_csv(f_open, index=False)
        self._status = 'finished'

    def predict(self, user_predictions: pd.DataFrame):
        '''
        Accepts and stores the user's predictions and unlocks iter_test once that is done
        '''
        if self._status == 'finished':
            raise Exception('You have already made predictions for the full test set.')
        if self._status != 'prediction_needed':
            raise Exception('You must get the next test sample from `iter_test()` first.')
        if not isinstance(user_predictions, pd.DataFrame):
            raise Exception('You must provide a DataFrame.')

        self.predictions.append(user_predictions)
        self._status = 'prediction_received'


def make_env():
    return MockApi()


### Функция для скора

In [ ]:
# Подсчитывает скор.
# Возвращает датафрейм compare со сравнением предсказаний
def calc_score():
    # Загружаем предсказания
    #submission = pd.read_csv(submission_name)
    submission = pd.concat(env.predictions)
    
    # Загружаем истинные значения
    revealed_targets = pd.read_csv(os.path.join(test_path, "revealed_targets.csv"))
    revealed_targets['data_block_id'] -= 2
    revealed_targets = revealed_targets[revealed_targets["data_block_id"] > train_end_data_block_id]
    # Обрезаем реальные предсказания revealed_targets по длине уже сделанных предсказаний submission
    revealed_targets = revealed_targets.iloc[:len(submission)]

    mae = mean_absolute_error(revealed_targets['target'] , submission['target'])
    # print(f'MAE: {mae}')
    
    # Подготовим данные для анализа изменения ошибки предсказания по мере удаления от времени завершения обучения
    compare = revealed_targets[['data_block_id', 'is_consumption', 'target']].copy()
    compare['predict'] = submission['target'].values
    compare['abs_err'] = abs(compare['predict'] - compare['target']).values
    compare['err'] = (compare['predict'] - compare['target']).values

    if (compare['data_block_id'].max() > 600):
        compare_600 = compare[compare['data_block_id'] > 600]
        # Выводим MAE для data_block_id > 600
        
        mae_600 = mean_absolute_error(compare_600['target'], compare_600['predict'])
        
        # Выводим MAE для data_block_id > 600 и is_consumption == 0
        mae_600_cons_0 = mean_absolute_error(compare_600[compare_600['is_consumption']==0]['target'], compare_600[compare_600['is_consumption']==0]['predict'])
        
        # Выводим MAE для data_block_id > 600 и is_consumption == 0
        mae_600_cons_1 = mean_absolute_error(compare_600[compare_600['is_consumption']==1]['target'], compare_600[compare_600['is_consumption']==1]['predict'])
    else:
        # Если еще не дошли до data_block_id > 600 не считаем эти величины
        mae_600, mae_600_cons_0, mae_600_cons_1 = '-', '-', '-'

    mae_df = pd.DataFrame({
        '(ALL)': mae,
        '(> 600)': mae_600,
        '(> 600, is_cons==0)': mae_600_cons_0,
        '(> 600, is_cons==1)': mae_600_cons_1
    }, index=['MAE'])

    # Округляем числа до двух знаков после запятой и преобразуем их в строки
    mae_df = mae_df.round(3).astype(str)
    
    display(mae_df)
    return compare

### Инициализация иттераций сабмита

In [ ]:
if is_local:
    # После этого можно имитировать локально загрузку при собмите на большом числе итераций
    # А не только четыре иттерации на 4 дня как в стандартной имитайии на кагле
    env = make_env()
else:
    # загружаем оригинальную библиотеку для сабмита
    import enefit
    env = enefit.make_env()

iter_test = env.iter_test()

### Цикл сабмита

In [ ]:
# Находим последний data_block_id в обучающих данных
max_train_data_block_id = df_data["data_block_id"].max()
# Устанавливаем первый data_block_id для теста следущим за тренировочным
cur_test_data_block_id = max_train_data_block_id + 1
cur_test_data_block_id

In [ ]:
%%time
count = 0

# Основной цикл для обработки данных тестового набора
for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
    print(p_time(), '*** Iteration: ', count, '***')
    # Переименование столбца для удобства
    test = test.rename(columns={"prediction_datetime": "datetime"})
    if is_local:
        # Если выполняем локально, то преобразуем некоторые типы данных
        # На кагле (а может и в линуксе) они и так преобразуются, но на виновс локально
        # не преобразуются и выдетают по ощибке
        test['datetime'] = pd.to_datetime(test['datetime'])
        client['date'] = pd.to_datetime(client['date'])
        gas_prices['origin_date'] = pd.to_datetime(gas_prices['origin_date'])
        gas_prices['forecast_date'] = pd.to_datetime(gas_prices['forecast_date'])
        electricity_prices['origin_date'] = pd.to_datetime(electricity_prices['origin_date'])
        electricity_prices['forecast_date'] = pd.to_datetime(electricity_prices['forecast_date'])
        forecast_weather['origin_datetime'] = pd.to_datetime(forecast_weather['origin_datetime'])
        forecast_weather['forecast_datetime'] = pd.to_datetime(forecast_weather['forecast_datetime'])
        historical_weather['datetime'] = pd.to_datetime(historical_weather['datetime'])
        revealed_targets['datetime'] = pd.to_datetime(revealed_targets['datetime'])
        
    # Добавляем колонку заполненную следующим data_block_id
    test["data_block_id"] = cur_test_data_block_id
    revealed_targets["data_block_id"] = cur_test_data_block_id
    
    df_test            = pl.from_pandas(test[data_cols[1:]], schema_overrides=schema_data)
    df_new_client      = pl.from_pandas(client[client_cols], schema_overrides=schema_client)
    df_new_gas         = pl.from_pandas(gas_prices[gas_cols], schema_overrides=schema_gas)
    df_new_electricity = pl.from_pandas(electricity_prices[electricity_cols], schema_overrides=schema_electricity)
    df_new_forecast    = pl.from_pandas(forecast_weather[forecast_cols], schema_overrides=schema_forecast)
    df_new_historical  = pl.from_pandas(historical_weather[historical_cols], schema_overrides=schema_historical)
    df_new_target      = pl.from_pandas(revealed_targets[target_cols], schema_overrides=schema_target)
    df_new_data        = pl.from_pandas(revealed_targets[df_data_cols], schema_overrides=schema_data)
    # Объединение новых данных с существующими и удаление дубликатов
    df_client          = pl.concat([df_client, df_new_client]).unique(subset=["county", "is_business", "product_type", "date"], maintain_order=True)
    df_gas             = pl.concat([df_gas, df_new_gas]).unique(subset=["forecast_date"], maintain_order=True)
    df_electricity     = pl.concat([df_electricity, df_new_electricity]).unique(subset=["forecast_date"], maintain_order=True)
    df_forecast        = pl.concat([df_forecast, df_new_forecast]).unique()
    df_historical      = pl.concat([df_historical, df_new_historical]).unique()
    df_target          = pl.concat([df_target, df_new_target]).unique()
    df_data            = pl.concat([df_data, df_new_data]).unique()
    
    
    '''
    if (count == 0):
        X, y = df_data.drop("target"), df_data.select("target")
        X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
        df_train = to_pandas(X, y)
        
        models.fit(df_train=df_train, itter_n=count)
    
    # Применение функции инженерии признаков и преобразование данных обратно в pandas
    X_test = feature_eng(df_test, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
    X_test = to_pandas(X_test)
    
    # Прогнозирование с использованием модели и ограничение предсказаний нулем
    #test['target'] = model.predict(X_test.drop(columns=["data_block_id"])).clip(0)
    test['target'] = models.predict(X_test)
    
    # Обновление целевых значений в примере предсказания
    sample_prediction["target"] = test['target']
    '''
    
    if (not(is_local) or (count == 0) or (count>=100)):
    #if (not(is_local) or (count>=0)):
        # Если испольняем локально только после
        # итерации 100. Потому что интересует как ведет себя моделья через два месяца
        # после обучения
        cur_time = time.time()
        if ((cur_time - notebook_starttime) < (8*60*60 + 60*30)):
            X, y = df_data.drop("target"), df_data.select("target")
            X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
            df_train = to_pandas(X, y)
            
            models.fit(df_train=df_train, itter_n=count)
        else:
            print('Не тренеруем модель, превышено время выполнения ноутбука:', (cur_time - notebook_starttime))

    # Применение функции инженерии признаков и преобразование данных обратно в pandas
    X_test = feature_eng(df_test, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target, working_days)
    X_test = to_pandas(X_test)
    
    # Прогнозирование с использованием модели и ограничение предсказаний нулем
    #test['target'] = model.predict(X_test.drop(columns=["data_block_id"])).clip(0)
    test['target'] = models.predict(X_test)
    
    # Обновление целевых значений в примере предсказания
    sample_prediction["target"] = test['target']
    
    # Отправка предсказаний в среду выполнения
    env.predict(sample_prediction)

    if is_local:
        # Выводим текущий скор в разных разрезах
        compare = calc_score()
    
    count += 1
    # Переходим к следующему data_block_id на итерации тестов
    cur_test_data_block_id += 1

## Анализ предсказания

### Подсчет скора

In [ ]:
if is_local:
    compare = calc_score()

### График MAE по дням предсказания

In [ ]:
# выводит график средних ошибок сгруппированных по дням (точнее для блоков данных для предсказаний которые в целом эквиваленты дням)
def print_err(err_name, err_lable, err_title):
    # Группируем по data_block_id, то есть по дням и считаем отдельно для каждого дня предсказания MAE
    grouped_compare = compare.groupby('data_block_id').mean().reset_index()
    # Делаем скользящую среднюю
    grouped_compare['rolling_mean'] = grouped_compare[err_name].rolling(window=30, min_periods=1).mean()
    
    
    # Plotting the mean absolute errors
    plt.figure(figsize=(10, 8))
    #plt.bar(grouped_compare['data_block_id'], grouped_compare['abs_err'])
    plt.bar(grouped_compare['data_block_id'], grouped_compare[err_name], label=err_lable)
    plt.plot(grouped_compare['data_block_id'],
             grouped_compare['rolling_mean'],
             label='Rolling Mean (window=30)',
             color='orange',
             linestyle='-', linewidth=2)
    plt.xlabel('data_block_id')
    plt.ylabel(err_lable)
    plt.title(err_title)
    plt.legend()
    
    # Set ticks every 10 data_block_id
    tick_positions_y = np.arange(-40, max(grouped_compare[err_name]) + 1, 10)
    plt.yticks(tick_positions_y)
    plt.grid(True)
    plt.show()

In [ ]:
if is_local:
    print_err(err_name='abs_err',
              err_lable='Mean Absolute Error',
              err_title='Mean Absolute Error by data_block_id')

In [ ]:
if is_local:
    compare = compare[compare["data_block_id"] > 600]
    print_err(err_name='err',
              err_lable='Mean Error (predict - target)',
              err_title='Mean Error by data_block_id')